# Differential Cryptanalysis of PHOTON<sub>256</sub>

# Importing necessary libraries

In [1]:
import random
import copy
from collections import deque
from IPython.core.display import HTML , Math
import galois
GF = galois.GF(2**4)
print(GF.properties)

Galois Field:
  name: GF(2^4)
  characteristic: 2
  degree: 4
  order: 16
  irreducible_poly: x^4 + x + 1
  is_primitive_poly: True
  primitive_element: x


In [2]:
%%HTML
<style type="text/css">
tr ,td {
    border: 1px  black solid !important;
  color: white !important;
}
</style>

# PHOTON<sub>256</sub> Permutation function

In [3]:
# ---------------------------------------- PHOTON-256-PERMUTATION ---------------------------------------------
## SBOX LIST
sbox_list = [0xc,5,6,0xb,9,0,0xa,0xd,3,0xe,0xf,8,4,7,1,2]
def list_64_to_8x8_matrix(s) -> list[list[int]]:
    assert len(s) == 64
    m = [[s[i+(j*8)] for j in range(8)] for i in range(8)]
    return m
def matrix_8x8_to_hex_list(m : list[list[int]]):
    lst = []
    for col in range(8):
        for row in range(8):
            lst.append(m[row][col])
    return lst
## 1.ADD-CONSTANT
def add_constant(X : list,k : list):
    new_X = copy.deepcopy(X)
    RC = [1, 3, 7, 14, 13, 11, 6, 12, 9, 2, 5, 10]
    IC = [0, 1, 3, 7, 15, 14, 12, 8]
    for i in range(8):
        new_X[i][0] = new_X[i][0] ^ RC[k] ^ IC[i]
    return new_X
## 2.SUB-CELL
def sub_cell(X):
    new_X = copy.deepcopy(X)
    for i in range(8):
        for j in range(8):
            new_X[i][j] = sbox_list[new_X[i][j]]
    return new_X
## 3.SHIFT-ROW
def shift_row(X):
    new_X = copy.deepcopy(X)
    for i in range(8):
        temp = deque(new_X[i])
        temp.rotate(-1*i)
        new_X[i] = list(temp)
    return new_X
# def shift_row(X):
#     new_X = copy.deepcopy(X)
#     for i in range(8):
#         temp = deque(new_X[i])
#         temp.rotate(1*i)
#         new_X[i] = list(temp)
#     return new_X
## 4.MIX-COLUMN-SERIAL
def serial(lst):
    M = []
    for i in range(7):
        a = [0 for j in range(8)]
        a[i+1] = 1
        M.append(a)
    M.append(copy.deepcopy(lst))
    return M
def matrix_mul(m1,m2):
    new_m = [[0 for j in range(8)] for i in range(8)]
    for i in range(8):
        for j in range(8):
            s = 0
            for temp in range(8):
                s ^= int(GF(m1[i][temp]) * GF(m2[temp][j]))
            new_m[i][j] = s
    return new_m
def mix_column_serial(X):
    new_X = copy.deepcopy(X)
    M = serial([2, 4, 2, 11, 2, 8, 5, 6])
    M8 = matrix_mul(M,M)
    for i in range(6):
        M8 = matrix_mul(M8,M)
    new_X = matrix_mul(M8,new_X)
    return new_X
## PHOTON 256 PERMUTATION FUNCTION
def PHOTON_256(input_hex_str = "0123456789abcdef0123456789abcdef0123456789abcdef0123456789abcdef"):
    if type(input_hex_str) == "":
        input_hex_str = [int(i,16) for i in input_hex_str]
    assert len(input_hex_str) == 64
    X = list_64_to_8x8_matrix(input_hex_str)
    # 0 to 11
    for i in range(12):
        X = add_constant(X,i)
        X = sub_cell(X)
        X = shift_row(X)
        X = mix_column_serial(X)
    X = matrix_8x8_to_hex_list(X)
    assert len(X) == 64
    return X

# Utility Functions to visualize propagation of difference

In [4]:
def get_coord(bit_pos):
    bit_pos -= 1
    # RETURNS POSITION
    box_num = (bit_pos//4) + 1
    bit_pos_in_4 = int(bit_pos % 4)
    x = (box_num % 8) - 1
    y = box_num // 8
    return [(x,y),bit_pos_in_4]
def int_to_bin(a,total_length):
    return list(map(int,format(a, f'0{total_length}b')))
def bin_list_to_int(lst):
    a = "".join(map(str,lst))
    a = int(a,2)
    return a
def flip_bits(state,bit_flip_positions=[]):
    m = copy.deepcopy(state)
    for i in bit_flip_positions:
        (x,y),bit_position = get_coord(i)
        a = int_to_bin(m[x][y],4)
        a[bit_position] = (a[bit_position] + 1) % 2
        a = bin_list_to_int(a)
        m[x][y] = a
    return m
def flip_boxes(m,box_nums=[]):
    new_m = copy.deepcopy(m)
    for pos in box_nums:
        x = (pos-1) % 8
        y = (pos-1) // 8
#         print(x,y)
        new_m[x][y] = new_m[x][y] ^ 1
    return new_m
def matrix_xor(m1,m2):
    l = len(m1[0])
    new_m = [[0 for i in range(l)] for j in range(l)]
    for i in range(l):
        for j in range(l):
            new_m[i][j] = m1[i][j] ^ m2[i][j]
    return new_m
def get_matrix_str(array,title=""):
    matrix = ''
    for row in array:
        try:
            for number in row:
                matrix += f'{number}&'
        except TypeError:
            matrix += f'{row}&'
        matrix = matrix[:-1] + r'\\'
    matrix = r'\begin{bmatrix}'+matrix+r'\end{bmatrix}'
    titled_matrix = r"\begin{matrix}" +matrix+r"\\"+title+ r"\end{matrix}"
    return titled_matrix

def get_hex_matrix_str(array,title=""):
    matrix = ''
    for row in array:
        try:
            for number in row:
                # matrix += f'{hex(number).split("x")[1]}&'
                num = "{:02x}".format(number)
                matrix += f'{num}&'
        except TypeError:
            matrix += f'{row}&'
        matrix = matrix[:-1] + r'\\'
    matrix = r'\begin{bmatrix}'+matrix+r'\end{bmatrix}'
    titled_matrix = r"\begin{matrix}" +matrix+r"\\ "+title+ r"\\ {} \end{matrix}"
    return titled_matrix
def get_active_matrix_html_str(m):
    row_lst = []
    a = 30
    ri = 1
    for row in m:
        ci = 0
        lst = []
        for ele in row:
            if ele > 0 :
                lst.append(f"<td style='background-color:black;width:{a}px;height:{a}px;color:white'>{ci*8 + ri}</td>")
            else:
                lst.append(f"<td style='width:{a}px;height:{a}px;color:black'></td>")
            ci += 1
        ri += 1
        row_str = f"<tr>{''.join(lst)}</tr>"
        row_lst.append(row_str)
    table_str = f"<table>{''.join(row_lst)}</table>"
    return table_str
def show_active_boxes_of_matrix(m):
    table_str = get_active_matrix_html_str(m)
    display(HTML(table_str))
def print_matrix(array,title=""):
    display(Math(get_matrix_str(array,title)))
def visualize_each_round(states,r=2):
    pass
def visualize_difference_propagation(msg_a,bit_positions_to_flip = [1,2,3,4]):
#     msg_b = flip_bits(msg_a,bit_positions_to_flip)
    msg_b = flip_boxes(msg_a,bit_positions_to_flip)
    operations = {'AC' : add_constant,'SC' : sub_cell,'SR' : shift_row,'MC' : mix_column_serial}
    states = {}
    for i in range(12):
        for o in operations:
            code = f"{i}-{o}"
            if o == 'AC':
                msg_a = copy.deepcopy(add_constant(msg_a,i))
                msg_b = copy.deepcopy(add_constant(msg_b,i))
                states[code] = matrix_xor(msg_a,msg_b)
            else:
                msg_a = copy.deepcopy(operations[o](msg_a))
                msg_b = copy.deepcopy(operations[o](msg_b))
                states[code] = matrix_xor(msg_a,msg_b)
    return states

# Implementation of visualization

In [8]:
# [1,9,17,25,10,18,26,34,19,27,35,43,28,36,44,52,37,45,53,61,46,54,62,55,63,64,8,16,24,7,15,6]
# pos_m=[
#     1,5,6,7,8,
#     9,10,14,15,16,
#     17,18,19,23,24,
#     25,26,27,28,32,
#     33,34,35,36,37,
#     42,43,44,45,46,
#     51,52,53,54,55,
#     60,61,62,63,64,
# ]
pos_m = [1,2,3,4,5]

In [9]:
msg_a = [[random.randint(0,15) for i in range(8)] for j in range(8)]
temp = visualize_difference_propagation(msg_a,pos_m)

In [14]:
for idx,ele in enumerate(temp.values()):
    if idx % 4 == 0 or idx == 0:
        show_active_boxes_of_matrix(ele)

1,,,,,,,
2,,,,,,,
3,,,,,,,
4,,,,,,,
5,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,


1,,,,33,41,49,57
2,,,,34,42,50,58
3,,,,35,43,51,59
4,,,,36,44,52,60
5,,,,37,45,53,61
6,,,,38,46,54,62
7,,,,39,47,55,63
8,,,,40,48,56,64


1,9,17,25,33,41,49,57
2,10,18,26,34,42,50,58
3,,19,27,,43,51,
4,12,,28,36,44,,60
5,13,21,,37,45,53,
6,14,22,30,38,46,54,62
7,15,23,31,39,47,55,63
8,16,24,32,,48,56,64


1,9,17,25,,41,49,57
,10,18,26,34,42,50,58
3,11,19,27,35,43,51,59
4,12,20,28,36,44,52,60
,13,21,29,37,45,53,61
6,14,22,30,,46,54,62
7,15,23,31,39,47,55,63
,16,24,32,40,48,56,64


1,9,17,25,33,41,49,57
2,10,18,26,34,42,50,58
3,,19,27,35,43,51,59
4,12,20,28,36,44,52,60
,13,21,29,,,53,61
6,14,22,30,38,46,54,62
7,15,23,31,39,47,55,63
8,,24,32,40,48,56,64


...

1,9,17,25,,41,49,
2,10,18,,,42,50,58
,11,,27,35,43,51,59
4,12,20,28,36,44,52,60
5,13,21,29,37,45,53,61
6,,22,30,38,46,54,62
7,15,23,31,39,47,55,
8,16,24,32,40,48,,64


1,9,17,25,33,41,49,57
2,,18,,34,42,,58
3,11,19,27,35,43,51,59
4,12,20,28,36,44,,60
5,13,21,29,37,45,53,61
6,14,22,30,38,46,54,62
7,15,23,31,39,47,55,63
8,16,24,32,40,48,56,64


1,9,17,25,33,41,49,57
2,10,18,26,34,42,50,58
3,11,19,27,35,43,51,59
4,12,20,28,36,44,52,60
5,13,21,29,37,45,53,61
6,14,22,30,38,46,54,62
7,15,23,31,39,47,55,63
8,16,24,32,40,48,56,64


1,9,17,25,33,41,49,57
2,10,18,26,34,42,,58
3,11,19,27,35,43,51,59
4,12,20,28,36,44,52,60
5,13,21,,,45,53,61
6,14,22,30,38,46,54,
7,15,23,31,39,47,,63
8,16,24,32,40,48,56,64


1,9,17,,33,41,49,57
2,10,18,26,34,42,50,58
3,11,,27,35,43,51,59
4,12,20,28,36,44,52,60
5,13,21,29,37,45,53,61
6,14,22,30,38,46,,62
7,15,23,31,39,47,55,63
8,16,24,32,40,48,56,64


# - - - END - - -